In [1]:
import sys

import os
import matplotlib.pyplot as plt
import numpy as np
import os
from skimage.filters import gaussian, threshold_otsu
from skimage.transform import resize
from skimage import io
import pickle
from skimage.measure import regionprops
import pandas as pd

In [73]:
sourceDir='/data/xinyi/c2p/data/chromark'
segDir=os.path.join(sourceDir,'nuclear_masks')
featureDir=os.path.join(sourceDir,'features')
featureFileName='nuclear_features.csv'
# markerFileName='marker_labels.csv'
gh2axFileName='gh2ax_foci_feats.csv'
imgDir=os.path.join(sourceDir,'raw_data')
conditions=['controls','headneck','meningioma', 'glioma']
proteins=['cd16', 'cd3', 'cd4', 'cd8', 'gh2ax', 'lamin']

In [83]:
outSize=128
updateImgs=True
savename='pathCentered_'+str(outSize)
for condition_i in conditions:
    print(condition_i)
    segDir_i=os.path.join(segDir,condition_i)
    imgDir_i=os.path.join(imgDir,condition_i)
    featureDir_i=os.path.join(featureDir,condition_i)
    for stain in os.listdir(segDir_i):
#         if condition_i=='headneck' and stain=='dapi_gh2ax_lamin_cd3': #skipping incorrect images
#             continue
        print(stain)
        segDir_i_stain=os.path.join(segDir_i,stain)
        imgDir_i_stain=os.path.join(imgDir_i,stain)
        featureDir_i_stain=os.path.join(featureDir_i,stain)
        
        segPID2name={}
        for pID_dir in os.listdir(segDir_i_stain):
            pID=pID_dir.split('_')
            segPID2name[pID[0]]=pID_dir
        imgPID2name={}
        for pID_dir in os.listdir(imgDir_i_stain):
            pID=pID_dir.split('_')
            imgPID2name[pID[0]]=pID_dir
        for pID in segPID2name.keys():
            if condition_i=='meningioma' and stain=='dapi_gh2ax_lamin_cd3' and pID=='P33': #skipping incorrect images
                continue
            print(pID)
            if pID not in imgPID2name:
                print('img not found '+pID)
                continue
            imgDir_i_stain_p=os.path.join(imgDir_i_stain,imgPID2name[pID])
            segDir_i_stain_p=os.path.join(segDir_i_stain,segPID2name[pID])
            featureDir_i_stain_p=os.path.join(featureDir_i_stain,segPID2name[pID])
            if os.path.exists(os.path.join(imgDir_i_stain_p,'nmco_allfeatures')) and updateImgs==False:
                print('already processed')
                continue
            nmco_all=pd.read_csv(os.path.join(featureDir_i_stain_p,featureFileName),index_col=0)
            if 'gh2ax' in stain:
                gh2ax_all=pd.read_csv(os.path.join(featureDir_i_stain_p,gh2axFileName),index_col=0)
#             else:
#                 marker_all=pd.read_csv(os.path.join(featureDir_i_stain_p,markerFileName),index_col=0)
            
            nmco_all_imgName=[]
            for f in range(nmco_all.shape[0]):
                nmco_all_imgName.append(nmco_all.index[f].rsplit('_',1)[0])
            nmco_all_imgName=np.array(nmco_all_imgName)
            
            dapiCol=np.repeat(True,nmco_all.shape[1])
            proteinCol={}
            for p in stain.split('_')[1:]:
                proteinCol[p]=np.repeat(False,nmco_all.shape[1])
                for c in range(nmco_all.columns.size):
                    if p in nmco_all.columns[c]:
                        proteinCol[p][c]=True
                        dapiCol[c]=False
            if 'gh2ax' in stain.split('_')[1:] and np.sum(proteinCol['gh2ax'])==20:
                gh2ax_foci_count=np.zeros(nmco_all.shape[0])
                for cell_i in range(nmco_all.shape[0]):
                    gh2ax_foci_count[cell_i]=np.sum(gh2ax_all['image_index']==nmco_all.index[cell_i])
            
                nmco_all['gh2ax_foci_count']=gh2ax_foci_count
                dapiCol=np.repeat(True,nmco_all.shape[1])
                proteinCol={}
                for p in stain.split('_')[1:]:
                    proteinCol[p]=np.repeat(False,nmco_all.shape[1])
                    for c in range(nmco_all.columns.size):
                        if p in nmco_all.columns[c]:
                            proteinCol[p][c]=True
                            dapiCol[c]=False
            
            nmco_all_res=None
            protein_all_res={}
            for imgname in os.listdir(segDir_i_stain_p):
                if not os.path.exists(os.path.join(imgDir_i_stain_p,imgname)):
                    print('missing '+imgname)
                    continue
                 
                imgname=imgname.split('.')[0]
                if nmco_all_res is None:
                    nmco_all_res=nmco_all.to_numpy()[nmco_all_imgName==imgname][:,dapiCol]
                    for p in stain.split('_')[1:]:
                        protein_all_res[p]=nmco_all.to_numpy()[nmco_all_imgName==imgname][:,proteinCol[p]]
                else:
                    nmco_all_res=np.concatenate((nmco_all_res,nmco_all.to_numpy()[nmco_all_imgName==imgname][:,dapiCol]),axis=0)
                    for p in stain.split('_')[1:]:
                        protein_all_res[p]=np.concatenate((protein_all_res[p],nmco_all.to_numpy()[nmco_all_imgName==imgname][:,proteinCol[p]]),axis=0)
            with open(os.path.join(imgDir_i_stain_p,'nmco_allfeatures'), 'wb') as output:
                pickle.dump(nmco_all_res,output,pickle.HIGHEST_PROTOCOL)
            with open(os.path.join(imgDir_i_stain_p,'nmco_allfeatures_protein'), 'wb') as output:
                pickle.dump(protein_all_res,output,pickle.HIGHEST_PROTOCOL)
                
            if not os.path.exists(os.path.join(sourceDir,'nmco_allfeatures_names')):
                with open(os.path.join(sourceDir,'nmco_allfeatures_names'), 'wb') as output:
                    pickle.dump(nmco_all.columns[dapiCol],output,pickle.HIGHEST_PROTOCOL)
            for p in stain.split('_')[1:]:
                if not os.path.exists(os.path.join(sourceDir,'nmco_allfeatures_names_'+p)):
                    with open(os.path.join(sourceDir,'nmco_allfeatures_names_'+p), 'wb') as output:
                        pickle.dump(nmco_all.columns[proteinCol[p]],output,pickle.HIGHEST_PROTOCOL)




controls
dapi_gh2ax_lamin
HV5
HV4
HV2
HV10
HV1
HV7
HV9
HV8
HV3
HV6
dapi_cd8_cd4
HV1
HV8
HV6
HV7
HV4
HV5
HV3
HV2
headneck
dapi_gh2ax_lamin_cd3
P72
missing TileScan 1_Position 37.tif
missing TileScan 1_Position 33.tif
P63
P14
P44
P41
P56
P18
P55
P50
P24
dapi_cd16_cd4_cd8
P55
P56
P72
P14
P18
P41
P44
P63
P50
P24
meningioma
dapi_gh2ax_lamin_cd3
P27
P70
P83
P42
P59
P38
P48
missing TileScan 1_Position 40.tif
missing TileScan 1_Position 45.tif
missing TileScan 1_Position 41.tif
missing TileScan 1_Position 44.tif
missing TileScan 1_Position 39.tif
missing TileScan 1_Position 43.tif
missing TileScan 1_Position 42.tif
P37
P62
dapi_cd16_cd4_cd8
P42
P48
missing TileScan 2_Position 25.tif
missing TileScan 2_Position 34.tif
missing TileScan 2_Position 29.tif
missing TileScan 2_Position 31.tif
missing TileScan 2_Position 33.tif
missing TileScan 2_Position 32.tif
missing TileScan 2_Position 28.tif
missing TileScan 2_Position 30.tif
missing TileScan 2_Position 26.tif
missing TileScan 2_Position 27.tif
m